In [1]:
import psycopg2
import xml.sax

class fileHandler(xml.sax.ContentHandler):
    def __init__(self) :
        self.currentData = ""
        self.pubkey = ""
        self.title = ""
        self.journal = ""
        self.booktitle = ""
        self.year = ""
        self.author = ""
        self.authorList = []
        self.validTag = ["article","author","inproceedings","title","journal","year","booktitle"]
    #only process article and inproceedings
    def startElement(self, tag, attributes):
        if tag in self.validTag:
            self.currentData = tag
            if tag == "article" or tag == "inproceedings":
                self.pubkey = attributes["key"]
                self.title = ""
                self.journal = ""
                self.booktitle = ""
                self.year = ""
                self.author = ""
                self.authorList = []

    def endElement(self,tag):

        if tag in self.validTag:
            #read author into self.authorList
            if tag == "author":
                self.authorList.append(self.author)
                self.author = ""
            #insert into "article" table
            if tag=="article":
                    list1=[self.pubkey,self.title,self.journal,self.year]
                    
                    for i in range(len(list1)):
                          if list1[i]=="":
                            list1[i]=None
                    
                    cur.execute('INSERT INTO Article(pubkey,title,journal,year) VALUES (%s,%s,%s,%s)', list1)

            #insert into "inproceedings" table
            if tag=="inproceedings":
                list2=[self.pubkey,self.title,self.booktitle,self.year]
                
                for i in range(len(list2)):
                      if list2[i]=="":
                        list2[i]=None
                
                
                cur.execute('INSERT INTO Inproceedings(pubkey,title,booktitle,year)VALUES(%s,%s,%s,%s)', list2)

            #insert into "authorship" table
            if tag=="article" or tag=="inproceedings":
                authorSet = set(self.authorList)
                for elem in authorSet:
                         list3=(self.pubkey, elem)
                         cur.execute('INSERT INTO Authorship(pubkey,author) VALUES (%s,%s)',list3)
                
        self.currentData = ""


  #call each time when read character
    def characters(self, content):
          if self.currentData in self.validTag:
                if(self.currentData =="title"):
                    self.title = content

                if(self.currentData =="journal"):
                    self.journal = content

                if(self.currentData =="booktitle"):
                    self.booktitle = content

                if(self.currentData =="year"):
                    self.year = content

                if(self.currentData =="author"):
                    self.author = content



In [2]:
conn = psycopg2.connect(database="dblp", user="dblpuser",password = "aptx4869", host = "127.0.0.1", port = "5432")
print (" connect to the database")
#Create a cursor
cur=conn.cursor()

#Create tables
cur.execute('CREATE TABLE Article(pubkey varchar PRIMARY KEY, title varchar, journal varchar, year int)')

cur.execute('CREATE TABLE Inproceedings(pubkey varchar PRIMARY KEY, title varchar, booktitle varchar, year int)')

cur.execute('CREATE TABLE Authorship(pubkey varchar, author varchar, PRIMARY KEY(pubkey,author))')

print("create tables")

#parse the file
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_namespaces, 0)
handler=fileHandler()
parser.setContentHandler(handler)
parser.parse("dblp-2019-09-05.xml")
print ("success")
conn.commit()
cur.close()
conn.close()



Successfully connect to the database
Successfully create tables
success
